In [2]:
import pandas as pd 
# loading the dataset
df = pd.read_csv('housing.csv') 
# displaying the first 5 rows of the dataframe
df.head()

,Title,Address,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Year,Views,Area,Road,Road Width,Road Type,Build Area,Posted,Amenities
0,Flat System House On Sale At Budhanikantha : H...,"Budhanikantha, Budhanilkantha, Kathmandu",Kathmandu,90000000,6,3,2.0,10,West,2073.0,17,1-0-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,18-0-0-0 Aana,11 hours ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
1,21 Aana Beautiful House : House For Sale In Bu...,"Pasikot, Budhanilkantha, Kathmandu",Kathmandu,80000000,5,3,2.0,9,East,2073.0,26,0-21-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-14-0-0 Aana,12 hours ago,"['Parking', 'Garage', 'Fencing', 'Frontyard', ..."
2,Beautiful To Floor House : House For Sale In D...,"Dhapasi, Dhapasi, Kathmandu",Kathmandu,70000000,5,3,2.0,12,East,2071.0,154,1-1-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,10-1-0-0 Aana,2 days ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
3,Modern Architecture Bungalow House : House For...,"baluwatar, Baluwatar, Kathmandu",Kathmandu,130000000,6,4,3.0,9,West,2017.0,17,0-10-1-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-7-1-0 Aana,1 day ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
4,Modern Bungalow House : House For Sale In Golf...,"Golfutar, Golfutar, Kathmandu",Kathmandu,100000000,6,3,2.0,10,East,2070.0,59,0-19-2-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-14-2-0 Aana,2 days ago,"['Parking', 'Lawn', 'Garage', 'Balcony', 'Back..."


# Getting started with preprocessing of data

In [3]:
# dropping title,views,Road and posted column as they are not useful for analysis 
df = df.drop(['Title','Views','Posted','Road'],axis=1) 
df.head()

,Address,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Year,Area,Road Width,Road Type,Build Area,Amenities
0,"Budhanikantha, Budhanilkantha, Kathmandu",Kathmandu,90000000,6,3,2.0,10,West,2073.0,1-0-0-0 Aana,20 Feet,Blacktopped,18-0-0-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
1,"Pasikot, Budhanilkantha, Kathmandu",Kathmandu,80000000,5,3,2.0,9,East,2073.0,0-21-0-0 Aana,20 Feet,Blacktopped,0-14-0-0 Aana,"['Parking', 'Garage', 'Fencing', 'Frontyard', ..."
2,"Dhapasi, Dhapasi, Kathmandu",Kathmandu,70000000,5,3,2.0,12,East,2071.0,1-1-0-0 Aana,20 Feet,Blacktopped,10-1-0-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
3,"baluwatar, Baluwatar, Kathmandu",Kathmandu,130000000,6,4,3.0,9,West,2017.0,0-10-1-0 Aana,20 Feet,Blacktopped,0-7-1-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
4,"Golfutar, Golfutar, Kathmandu",Kathmandu,100000000,6,3,2.0,10,East,2070.0,0-19-2-0 Aana,20 Feet,Blacktopped,0-14-2-0 Aana,"['Parking', 'Lawn', 'Garage', 'Balcony', 'Back..."


In [4]:
# taking only the  city prefix from the address column as city covers the main information 
df['Address1'] = df['Address'].apply(lambda x: x.split(',')[0].strip()) 
df['Address2'] = df['Address'].apply(lambda x: x.split(',')[1].strip())  
# dropping the original address column 
df = df.drop(['Address'],axis=1) 
df.head()

,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Year,Area,Road Width,Road Type,Build Area,Amenities,Address1,Address2
0,Kathmandu,90000000,6,3,2.0,10,West,2073.0,1-0-0-0 Aana,20 Feet,Blacktopped,18-0-0-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe...",Budhanikantha,Budhanilkantha
1,Kathmandu,80000000,5,3,2.0,9,East,2073.0,0-21-0-0 Aana,20 Feet,Blacktopped,0-14-0-0 Aana,"['Parking', 'Garage', 'Fencing', 'Frontyard', ...",Pasikot,Budhanilkantha
2,Kathmandu,70000000,5,3,2.0,12,East,2071.0,1-1-0-0 Aana,20 Feet,Blacktopped,10-1-0-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe...",Dhapasi,Dhapasi
3,Kathmandu,130000000,6,4,3.0,9,West,2017.0,0-10-1-0 Aana,20 Feet,Blacktopped,0-7-1-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe...",baluwatar,Baluwatar
4,Kathmandu,100000000,6,3,2.0,10,East,2070.0,0-19-2-0 Aana,20 Feet,Blacktopped,0-14-2-0 Aana,"['Parking', 'Lawn', 'Garage', 'Balcony', 'Back...",Golfutar,Golfutar


In [5]:
# Converting the road_width column to numeric by removing 'Feet' and 'Meter' and converting to float 
df['Road Width'] = df['Road Width'].astype(str)

mask_feet = df['Road Width'].str.contains('Feet', case=False, na=False)
mask_meter = df['Road Width'].str.contains('Meter', case=False, na=False)

df.loc[mask_feet, 'Road Width'] = (
    df.loc[mask_feet, 'Road Width']
      .str.replace(r'[^0-9.]', '', regex=True)
      .astype(float)
)

df.loc[mask_meter, 'Road Width'] = (
    df.loc[mask_meter, 'Road Width']
      .str.replace(r'[^0-9.]', '', regex=True)
      .astype(float) * 3.281
)

df['Road Width(in feet)'] = pd.to_numeric(df['Road Width'], errors='coerce')
df = df.drop('Road Width',axis=1)
df.head()

,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Year,Area,Road Type,Build Area,Amenities,Address1,Address2,Road Width(in feet)
0,Kathmandu,90000000,6,3,2.0,10,West,2073.0,1-0-0-0 Aana,Blacktopped,18-0-0-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe...",Budhanikantha,Budhanilkantha,20.0
1,Kathmandu,80000000,5,3,2.0,9,East,2073.0,0-21-0-0 Aana,Blacktopped,0-14-0-0 Aana,"['Parking', 'Garage', 'Fencing', 'Frontyard', ...",Pasikot,Budhanilkantha,20.0
2,Kathmandu,70000000,5,3,2.0,12,East,2071.0,1-1-0-0 Aana,Blacktopped,10-1-0-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe...",Dhapasi,Dhapasi,20.0
3,Kathmandu,130000000,6,4,3.0,9,West,2017.0,0-10-1-0 Aana,Blacktopped,0-7-1-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe...",baluwatar,Baluwatar,20.0
4,Kathmandu,100000000,6,3,2.0,10,East,2070.0,0-19-2-0 Aana,Blacktopped,0-14-2-0 Aana,"['Parking', 'Lawn', 'Garage', 'Balcony', 'Back...",Golfutar,Golfutar,20.0


In [6]:
# Converting the year column into age of the house 
import datetime 
def current_bs_year():
    now = datetime.datetime.now()
    current_year_ad = now.year
    current_year_bs = current_year_ad + 57
    return current_year_bs 
df['Age'] = df['Year'].apply(lambda x: current_bs_year() - x)
df = df.drop('Year',axis=1)
df.head()

,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Area,Road Type,Build Area,Amenities,Address1,Address2,Road Width(in feet),Age
0,Kathmandu,90000000,6,3,2.0,10,West,1-0-0-0 Aana,Blacktopped,18-0-0-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe...",Budhanikantha,Budhanilkantha,20.0,9.0
1,Kathmandu,80000000,5,3,2.0,9,East,0-21-0-0 Aana,Blacktopped,0-14-0-0 Aana,"['Parking', 'Garage', 'Fencing', 'Frontyard', ...",Pasikot,Budhanilkantha,20.0,9.0
2,Kathmandu,70000000,5,3,2.0,12,East,1-1-0-0 Aana,Blacktopped,10-1-0-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe...",Dhapasi,Dhapasi,20.0,11.0
3,Kathmandu,130000000,6,4,3.0,9,West,0-10-1-0 Aana,Blacktopped,0-7-1-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe...",baluwatar,Baluwatar,20.0,65.0
4,Kathmandu,100000000,6,3,2.0,10,East,0-19-2-0 Aana,Blacktopped,0-14-2-0 Aana,"['Parking', 'Lawn', 'Garage', 'Balcony', 'Back...",Golfutar,Golfutar,20.0,12.0


In [7]:
import re
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import Normalizer

ROPANI_TO_SQFT = 5476
AANA_TO_SQFT = 342.25
PAISA_TO_SQFT = 85.56
DAAM_TO_SQFT = 21.39

def parse_nepali_area(area_str):
    if pd.isna(area_str) or str(area_str).strip() == "":
        return np.nan
    
    try:
        area_str = str(area_str).replace("Aana", "").strip()
        parts = area_str.split("-")
        if len(parts) != 4:
            return np.nan 
        
        R, A, P, D = map(int, parts)
        
        sqft = (R * ROPANI_TO_SQFT +
                A * AANA_TO_SQFT +
                P * PAISA_TO_SQFT +
                D * DAAM_TO_SQFT)
        return sqft
    except:
        return np.nan


for col in ['Area', 'Build Area']:
    if col in df.columns:
        new_col = col.replace(' ', '_') + '_sqft'
        df[new_col] = df[col].apply(parse_nepali_area)
        df[new_col] = pd.to_numeric(df[new_col], errors='coerce')
        print(f"{col}: parsed {df[new_col].notna().sum()} non-null values, {df[new_col].isna().sum()} NaNs")

df = df.drop([c for c in ['Area', 'Build Area'] if c in df.columns], axis=1)

Area: parsed 663 non-null values, 1548 NaNs
Build Area: parsed 56 non-null values, 2155 NaNs


In [8]:
df["Area_sqft"] = df["Area_sqft"].fillna(df["Area_sqft"].median())
df["Build_Area_sqft"] = df["Build_Area_sqft"].fillna(df['Area_sqft'])

In [9]:
# Dropping anemities column as it has too many unique values and not useful for analysis 
df = df.drop('Amenities',axis=1) 
df.head()

,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Road Type,Address1,Address2,Road Width(in feet),Age,Area_sqft,Build_Area_sqft
0,Kathmandu,90000000,6,3,2.0,10,West,Blacktopped,Budhanikantha,Budhanilkantha,20.0,9.0,5476.00,98568.00
1,Kathmandu,80000000,5,3,2.0,9,East,Blacktopped,Pasikot,Budhanilkantha,20.0,9.0,7187.25,4791.50
2,Kathmandu,70000000,5,3,2.0,12,East,Blacktopped,Dhapasi,Dhapasi,20.0,11.0,5818.25,55102.25
3,Kathmandu,130000000,6,4,3.0,9,West,Blacktopped,baluwatar,Baluwatar,20.0,65.0,3508.06,2481.31
4,Kathmandu,100000000,6,3,2.0,10,East,Blacktopped,Golfutar,Golfutar,20.0,12.0,6673.87,4962.62


In [10]:
# removing outliers from price column 
Q1 = df['Price'].quantile(0.25) 
Q3 = df['Price'].quantile(0.75) 
IQR = Q3 - Q1 
lower_bound = Q1 - 1.5 * IQR 
upper_bound = Q3 + 1.5 * IQR 
df = df[(df['Price'] >= lower_bound) & (df['Price'] <= upper_bound)]
df.head()

,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Road Type,Address1,Address2,Road Width(in feet),Age,Area_sqft,Build_Area_sqft
6,Lalitpur,11000000,3,4,3.0,1,South East,Gravelled,Dadathok,Lubhu,11.0,63.0,1711.25,1711.25
7,Kathmandu,50000000,5,4,2.0,9,East,Blacktopped,Hepali,Budhanilkantha,20.0,9.0,3165.80,2481.30
9,Kathmandu,32500000,6,4,2.5,1,South East,Concrete,Nearby Nilo Pool,Sukedhara,12.0,17.0,1369.00,1369.00
10,Pokhara,32000000,4,5,3.0,3,South West,Blacktopped,4,Lekhnath,15.0,7.0,1711.25,1711.25
11,Kathmandu,39900000,5,4,3.0,1,North West,Paved,Budhanilkantha,Chapali,14.0,7.0,1540.12,1540.12


In [11]:
# making groups for checking if address2 is of address 1 or not 
from collections import defaultdict

df['Address1'] = df['Address1'].astype(str)
df['Address2'] = df['Address2'].astype(str)
df['Address1'] = df['Address1'].str.lower() 
df['Address2'] = df['Address2'].str.lower() 
df['Address1'] = df['Address1'].apply(lambda x:x.title())
df['Address2'] = df['Address2'].apply(lambda x:x.title())
checking_address1 = defaultdict(list)
checking_address2 = defaultdict(list)

for a1, a2 in zip(df['Address1'], df['Address2']):
    if a2 not in checking_address1[a1]:
        checking_address1[a1].append(a2)
    if a1 not in checking_address2[a2]:
        checking_address2[a2].append(a1)
# checking the groups 
checking_address1 = dict(checking_address1) 
checking_address2 = dict(checking_address2) 
checking_address1,checking_address2

({'Dadathok': ['Lubhu'],
  'Hepali': ['Budhanilkantha', 'Golfutar'],
  'Nearby Nilo Pool': ['Sukedhara'],
  '4': ['Lekhnath'],
  'Budhanilkantha': ['Chapali',
   'Budhanilkantha',
   'Opposite Park Village Resort',
   'Taulung.',
   '100 Mtrs From Main Road',
   'Narayanthan',
   'Pasikot',
   'Mahankal',
   'Muhanpokhari',
   'Hattigauda'],
  'Na': ['Budhanilkantha',
   'Sitapaila',
   'Dhapasi',
   'Pepsicola',
   'Kapan',
   'Mandikatar',
   'Hattigauda',
   'Tokha',
   'Sukedhara',
   'Golfutar',
   'Bishal Nagar',
   'Baluwatar',
   'Ranibari',
   'Hattisar',
   'Sanobharyang',
   'Swayambhu',
   'Chapali',
   'Maharajgunj',
   'Pasikot',
   'Samakhusi',
   'Chabahil',
   'Baniyatar',
   'Basundhara',
   'Bhangal',
   'Dhungedhara',
   'Naya Bazar',
   'Thulo Bharyang',
   'Dhumbarahi',
   'Jorpati',
   'Grande Villa',
   'Ekatabasti',
   'Thali'],
  'Nearby Ccrc College': ['Narephate'],
  'Daura Dipo': ['Thankot'],
  'Khasibazaar': ['Kalanki'],
  'Lonag Height': ['Kavresthali'],


In [12]:
# making groups for checking address1 belongs to city or not 
checking_city = defaultdict(list)
df['City'] = df['City'].astype(str) 
df['City'] = df['City'].str.lower() 
df['City'] = df['City'].apply(lambda x:x.title())
for c,a1 in zip(df['City'],df['Address1']):
    if a1 not in checking_city[c]:
        checking_city[c].append(a1) 
checking_city = dict(checking_city) 
checking_city

{'Lalitpur': ['Dadathok',
  'Sital Height',
  'Bojepokhari',
  'Swastik Tole',
  'Nakkhudol Bhaisepati',
  'Nearby Cozy Homes',
  'Sanagaun Height',
  'Nearby',
  'Imadol',
  'Imadole',
  'Opposite Of Green Hill City',
  'Nearby Pawan Pakriti School.',
  'Ochu Height',
  'Kist Hospital To Hatiban Road',
  'Imadol Mahalaxmi 1',
  'Sitapakha',
  'Sanepa',
  'Jhamsihel (Near British School)',
  'Velpa Hattiban',
  'Janta Awash Bhaisepati',
  'Hattiban',
  'Manbhawan',
  'Boje Pokhari',
  'Patan',
  'Jhamsikhel',
  'Tikathali',
  'Badikhel (Godawari)',
  'Nearby Panityanki',
  'Lubhu Lamatar',
  'Nearby Buddha Party Palace',
  'Pulchowk',
  'Bhaisepati',
  'Harisiddhi',
  'Talchhikhel',
  'Dhapakhel Height',
  'Jawalakhel',
  'Enar Chowk',
  'Bojopokhari',
  'Champi',
  'Taukhel',
  'House Rent In Bagdol',
  'Sainbu',
  '27.6564024',
  'Jhamsikhel(Near British School)',
  'Lele',
  'Nearby Sumeru Hospital',
  'Kumaripati',
  'Kupondole Heights Road',
  'Bakhundole Sanepa',
  'Smritinagar',

In [13]:
# encoding categorical columns using label encoding
from sklearn.preprocessing import LabelEncoder 
categorical_cols = ['City','Face','Road Type','Address1','Address2'] 
encoders = {}
encoded_values = {}
for col in categorical_cols:
    le = LabelEncoder() 
    df[col] = le.fit_transform(df[col].astype(str)) 
    encoded_values[col] = list(le.classes_)
    encoders[col] = le 

In [14]:
# check for null values 
df.isnull().sum()

City                      0
Price                     0
Bedroom                   0
Bathroom                  0
Floors                 1131
Parking                   0
Face                      0
Road Type                 0
Address1                  0
Address2                  0
Road Width(in feet)       0
Age                    1555
Area_sqft                 0
Build_Area_sqft           0
dtype: int64

In [15]:
imputer = KNNImputer(n_neighbors=5) 
imputed_arr=imputer.fit_transform(df) 
df = pd.DataFrame(imputed_arr, columns=df.columns)
df.isnull().sum()

City                   0
Price                  0
Bedroom                0
Bathroom               0
Floors                 0
Parking                0
Face                   0
Road Type              0
Address1               0
Address2               0
Road Width(in feet)    0
Age                    0
Area_sqft              0
Build_Area_sqft        0
dtype: int64

In [16]:
# normalizing the numerical columns 
numerical_cols = ['Bedroom','Bathroom','Parking','Area_sqft','Build_Area_sqft','Age','Road Width(in feet)']
normalizer = Normalizer()
df[numerical_cols] = normalizer.fit_transform(df[numerical_cols]) 
df.head()

,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Road Type,Address1,Address2,Road Width(in feet),Age,Area_sqft,Build_Area_sqft
0,19.0,11000000.0,0.001239,0.001652,3.0,0.000413,5.0,3.0,262.0,225.0,0.004544,0.026023,0.706858,0.706858
1,16.0,50000000.0,0.001243,0.000994,2.0,0.002237,0.0,1.0,433.0,70.0,0.004972,0.002237,0.787042,0.616870
2,16.0,32500000.0,0.003099,0.002066,2.5,0.000516,5.0,2.0,819.0,376.0,0.006198,0.008780,0.707061,0.707061
3,26.0,32000000.0,0.001653,0.002066,3.0,0.001240,6.0,1.0,18.0,220.0,0.006198,0.002892,0.707087,0.707087
4,16.0,39900000.0,0.002296,0.001836,3.0,0.000459,3.0,4.0,186.0,82.0,0.006428,0.003214,0.707085,0.707085


In [17]:
df.columns

Index(['City', 'Price', 'Bedroom', 'Bathroom', 'Floors', 'Parking', 'Face',
       'Road Type', 'Address1', 'Address2', 'Road Width(in feet)', 'Age',
       'Area_sqft', 'Build_Area_sqft'],
      dtype='object')

In [18]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = df.drop('Price', axis=1)
y = scaler.fit_transform(df['Price'].values.reshape(-1,1))
X_num = X.select_dtypes(include=[np.number]).fillna(0)

if X_num.shape[0] < 2:
    raise ValueError("Not enough rows to perform train/test split.")

X_train, X_test, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=42)

rf_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"]
}
rf_gs = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid,
                     cv=5, scoring="neg_root_mean_squared_error", n_jobs=-1, verbose=1)
rf_gs.fit(X_train, y_train)
rf_best = rf_gs.best_estimator_
rf_pred = rf_best.predict(X_test)

rf_rmse = root_mean_squared_error(y_test, rf_pred)  
rf_r2 = r2_score(y_test, rf_pred)

print("RF best params:", rf_gs.best_params_)
print('RF best estimator: ',rf_best)
print('RF best score: ', rf_gs.best_score_)
print("RF test RMSE:", round(rf_rmse, 4), " R2:", round(rf_r2, 4))
lr_param_grid = {
    "fit_intercept": [True, False],
    "positive": [False, True]
}
lr_gs = GridSearchCV(LinearRegression(), lr_param_grid,
                     cv=5, scoring="neg_root_mean_squared_error", n_jobs=-1, verbose=1)
lr_gs.fit(X_train, y_train)
lr_best = lr_gs.best_estimator_
lr_pred = lr_best.predict(X_test)

lr_rmse = root_mean_squared_error(y_test, lr_pred)
lr_r2 = r2_score(y_test, lr_pred)

print("LR best params:", lr_gs.best_params_)
print('LR best estimator: ',lr_best)
print('LR best score: ', lr_gs.best_score_)
print("LR test RMSE:", round(lr_rmse, 4), " R2:", round(lr_r2, 4))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


c:\Users\Sakriya Pyakurel\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RF best params: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 200}
RF best estimator:  RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=200,
                      random_state=42)
RF best score:  -0.1511911331283593
RF test RMSE: 0.1496  R2: 0.6313
Fitting 5 folds for each of 4 candidates, totalling 20 fits
LR best params: {'fit_intercept': False, 'positive': False}
LR best estimator:  LinearRegression(fit_intercept=False)
LR best score:  -0.19166190666653252
LR test RMSE: 0.1889  R2: 0.4122


<b>Interpretation:</b> Random forest regressor has highest r_2 score of 0.6313 so we will choose random forest regressor with the best choosen parameters for training our model

In [19]:
# training the final model using random forest regressor 
final_model = RandomForestRegressor(**rf_gs.best_params_, random_state=42) 
final_model.fit(X_train,y_train) 
if final_model is not None:
    print('Final model trained successfully') 
else:
    print('Final model training failed')

c:\Users\Sakriya Pyakurel\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Final model trained successfully


In [20]:
# storing the model and other necessary objects using pickle 
import pickle 
with open('final_model.pkl', 'wb') as f:
    pickle.dump({
        'model': final_model,
        'encoders': encoders,
        'scaler':scaler,
        'numerical_cols': numerical_cols,
        'categorical_cols': categorical_cols,
        'address1_mapping': checking_address1,
        'address2_mapping':checking_address2,
        'city_mapping':checking_city,
        'ordering':X_train.columns.tolist()
    }, f)